结合REINFORCE-DQN的优势创建名为演员-评论家模型的算法。

REINFORCE通常作为一种情景式算法来实现，这意味着只有智能体完成整个轮次（并在此过程中收集奖励）后才会使用它来更新模型参数。

通过对完整的轮次进行抽样，我们能够更好的了解动作的真正价值，因为这样能够看到其下游影响而非仅仅是直接影响（由于环境的随机性，这可能会产生误导），这个完整的轮次抽烟属于蒙特卡洛方法的范围。然而并非所有环境都是情景式的，有时我们希望能够以增量方式或在线方式进行更新，例如，无论环境中发生什么都定期进行更新。DQN在非情景环境中表现良好，可以将其视为在线学习算法，但为了有效的学习，他需要经验回放缓冲器。

由于环境的内在变化，真正的在线学习（在每个动作之后进行参数更新）都是不稳定的，因此回放缓冲器非常必要。一个动作偶尔可能会导致较大的负向奖励，但从预期来看（平均长期奖励）他可能是一个好动作--单个动作之后的更新可能会导致错误的参数更新，从而最终阻碍充分的学习。

分布式优势演员-评论家（DA2C）的新型策略梯度法。该算法具有DQN在线学习的优势，无须回放缓冲器。他还具有策略算法的优点，即可以直接从动作的概率分布中对动作进行抽样，从而消除了DQN中选择策略的需要。


Q-learning 的伟大之处在于他直接根据环境中可用的信息（奖励）进行学习。他基本上学会了预测奖励，也就是我们所说的价值。

注：价值函数（状态-价值或动作-价值）隐式依赖于策略的选择，所以应该写出$V_{\pi}(S)$来予以明示。但是为了符号简洁性，我们去掉了下标$\pi$。策略对价值的影响至关重要，因为总是采取随机动作的策略导致所有状态的价值差不多同样低

演员-评论家算法：演员是指策略，因为那是动作产生的地方，评论家是指价值函数，因为它（部分）告诉演员其动作的好坏。演员预测最佳动作并选择要采取的动作，然后生成新的状态。评论家网络计算旧状态和新状态的价值。$S_{t+1}$的相应的值称为其优势，该信号将用于强化演员采取的动作。

由于使用R-V(S)而不仅仅是V(S)来训练策略，因此称为优势演员-评论家算法

Q-learning 属于价值算法的范畴，因为试图学习的是动作价值，而像REINFORCE这样的策略梯度法则直接尝试学习要采取的最佳动作。我们可以将这两种组合成所谓的演员-评论家算法

策略函数有一个敏感的损失函数，他依赖于轮次结束时收集到的奖励--解决办法：自举

偏差-方差权衡：表示然后机械学习模型都会与真实的数据分布存在某种程度上的系统偏差和方差。可以尝试减小模型的方差，但这样会导致偏差增大。增加模型复杂性可以减小偏差但会导致方差增大，而减小方差则会导致偏差增大。

对于大部分深度学习模型，我们都会进行分批训练，即将训练数据的一个随机子集进行分批，然后在反向传播和梯度下降处理之前计算整个批次的损失。这是很有必要的，因为如果每次都使用单块数据进行训练，那么梯度将包含太多变化，且参数将永远不会收敛到他们的最优值。在更新模型参数之前，我们需要对一批数据中的噪声进行平均来得到真实的信号。

一般的RNN层通过将之前的输出与新的输入合并来处理一组数据，随后新产生的输出在下个时间步重新输入RNN，而其副本可能被输入另一层。RNN无法利用经验回放缓冲器中的单条经验来正常工作，因为他需要处理一系列经验。

经验回放不能与RNN一起工作，除非回放缓冲器存储来整个轨迹或完整的轮次，因为RNN是用来处理序列数据的

一种没有经验回放的情况下使用RNN的方法是并行运行智能体的多个副本，每个副本都有单独的环境实例。通过将多个独立的智能体分布到不同的CPU进程中，我们可以获得各种各样的经验，因此能够得到梯度的样本，可以对这些梯度共同求取平均值来得到一个方差较小的平均梯度。这消除了对经验回放的需求，并允许我们以完全的方式训练算法，且每个状态在环境中出现时只会访问一次。

深度学习模型比较常见的训练形式时将一批数据一起输入模型中来返回一批预测，然后计算每个预测的损失，并在反向传播和更新模型参数之前对所有损失进行平均或求和，这就平均掉了所有经验中存在的变动性。也可以运行多个模型，每个模型接收一条经验并做出单个预测，通过每个模型反向传播得到梯度，然后在进行任何参数更新之前对这些梯度进行平均或求和。

Python 提供了一个叫multiprocessing的库，使得多进程操作变得非常容易。此外，pytorch 封装了这个库，并提供了一个方法来实现模型的参数在多个进程之间共享。

多进程示例：

In [20]:
import multiprocessing as mp
import numpy as np
def square(x): #这个函数接收一个数组并对其中的元素求平方
    return np.square(x)
x = np.arange(64) #设置一个包含数字序列的数组
print(x)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63]


In [21]:
#mp.cpu_count()  用于返回计算机上可用的CPU核心数量。这个数量可以用来决定创建多少个进程是合理的，以避免过度消耗系统资源。
mp.cpu_count()

8

许多现代计算机可能拥有 4 个独立的物理处理器，但他们通过超线程拥有 8 个虚拟处理器。

超线程是某些处理器使用的一种性能技巧，可以让两个进程本质上同时运行与一个物理器上。

注：不要在计算机上创建多于CPU数量的进程数，因为额外的进程实质上将作为线程来运行，而CPU将不得不在进程间快速切换

演员-评论家算法产生状态-价值和动作概率用于计算优势，它是用来训练模型的量，而不是像Q-learning那样的原始奖励

创建演员-评论家模型（一个双头模型，也可以分别创建独立的演员和评论家网络）。

当处于当前轮次时，所做的操作如下：

a、定义超参数：$\gamma$（贴现因子）

b、在初始状态$s_{t}$中启动一个新的轮次

c、计算值$v(s_{t})$并将其存储在列表中

d、计算$\pi(s_{t})$并将其存储在列表中，然后抽样并采取动作$a_{t}$。获得新的状态$s_{t+1}$ 和奖励$r_{t+1}$，并将奖励存储在列表中

训练：

a、初始化R=0。逆序遍历奖励来产生回报：$R=r_{i}+\gamma R$

b、最小化演员损失：$-1*\gamma_{t}(R-v(s_{t}))\pi(a|s)$

c、最小化评论家损失：$(R-v)^{2}$

重复一个新的轮次

针对CartPole编写演员-评论家算法

In [ ]:
import torch
from torch import nn
from torch import optim
import numpy as np
from torch.nn import functional as F
import gym
import torch.multiprocessing as mp #pytorch封装了 Python内置的multiprocessing库，且API相同

class ActorCritic(nn.Module): #为演员和评论家定义一个组合模型
    def __init__(self):
        #调用父类 nn.Module 的构造函数，确保正确初始化
        super(ActorCritic, self).__init__()
        #网络架构：共享层（l1、l2）、策略网络（actor_lin1）、价值网络（l3、critic_lin1）
        self.l1 = nn.Linear(4,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,2)  #输出动作概率分布
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)   #输出状态价值估计
    def forward(self,x):
        #对输入状态 x 沿维度 0 进行归一化处理
        x = F.normalize(x,dim=0)
        #将归一化后的状态通过共享层l1和l2，使用relu激活函数u
        y = F.relu(self.l1(x))
        y = F.relu(self.l2(y))
        #log_softmax逻辑上等同于log(softmax(..))，但组合后的函数数值上更稳定。如果单独计算函数，则在softmax之后可能会得到满溢或下溢概率
        actor = F.log_softmax(self.actor_lin1(y),dim=0) #演员这一头返回两个动作的对数概率
        #使用 y.detach() 创建一个分离的副本，阻止梯度从Critic流向共享层，
        #所以评论家损失不会在第一层和第二层中反向传播和修改权重。
        #只有演员才会导致这些权重被修改，所以当演员和评论家试图对前面的网络层做出相反的更新时，这能够防止二者之间的冲突
        c = F.relu(self.l3(y.detach()))
        critic = torch.tanh(self.critic_lin1(c)) #评论家返回一个以-1 和+1 位界限的数字
        return actor, critic #以元组形式返回演员和评论家结果

In [ ]:

def run_episode(worker_env, worker_model):
    state = torch.from_numpy(worker_env.env.state).float() 
    values, logprobs, rewards = [],[],[] 
    done = False
    j=0
    while (done == False): 
        j+=1
        policy, value = worker_model(state) 
        values.append(value)
        logits = policy.view(-1)
        action_dist = torch.distributions.Categorical(logits=logits)
        action = action_dist.sample() 
        logprob_ = policy.view(-1)[action]
        logprobs.append(logprob_)
        state_, _, done, _, info = worker_env.step(action.detach().numpy())
        state = torch.from_numpy(state_).float()
        if done: #F
            reward = -10
            worker_env.reset()
        else:
            reward = 1.0
        rewards.append(reward)
    return values, logprobs, rewards

In [ ]:

def update_params(worker_opt,values,logprobs,rewards,clc=0.1,gamma=0.95):
        rewards = torch.Tensor(rewards).flip(dims=(0,)).view(-1) 
        logprobs = torch.stack(logprobs).flip(dims=(0,)).view(-1)
        values = torch.stack(values).flip(dims=(0,)).view(-1)
        Returns = []
        ret_ = torch.Tensor([0])
        for r in range(rewards.shape[0]): #B
            ret_ = rewards[r] + gamma * ret_
            Returns.append(ret_)
        Returns = torch.stack(Returns).view(-1)
        Returns = F.normalize(Returns,dim=0)
        actor_loss = -1*logprobs * (Returns - values.detach()) #C
        critic_loss = torch.pow(values - Returns,2) #D
        loss = actor_loss.sum() + clc*critic_loss.sum() #E
        loss.backward()
        worker_opt.step()
        return actor_loss, critic_loss, len(rewards)

In [5]:
def worker(t, worker_model, counter, params):
    worker_env = gym.make("CartPole-v1")
    worker_env.reset()
    worker_opt = optim.Adam(lr=1e-4,params=worker_model.parameters()) #每个进程运行自己独立的环境和优化器，但共享模型
    worker_opt.zero_grad()
    for i in range(params['epochs']):
        worker_opt.zero_grad()
        #run_episode函数将运行游戏的一个轮次，并沿途收集数据
        values, logprobs, rewards = run_episode(worker_env,worker_model) 
        actor_loss,critic_loss,eplen = update_params(worker_opt,values,logprobs,rewards) #C
        counter.value = counter.value + 1 #D

In [6]:
MasterNode=ActorCritic()
MasterNode.share_memory
processes=[]
params={'epochs':1000,'n_workers':7,}
counter=mp.Value('i',0)
for i in range(params['n_workers']):
    p=mp.Process(target=worker,args=(i,MasterNode,counter,params))
    p.start()
    processes.append(p)
for p in processes:
    p.join()
for p in processes:
    p.terminate()

print(counter.value,processes[1].exitcode)

0 1


Traceback (most recent call last):
Traceback (most recent call last):
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=77, pipe_handle=80)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/rl/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/anaconda3/envs/rl/lib/python3.13/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
  File "<string>", line 1, in <module>
    from multiprocessing.spawn import spawn_main; spawn_main(tracker_fd=77, pipe_handle=82)
                                                  ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/rl/lib/python3.13/multiprocessing/spawn.py", line 122, in spawn_m